# 라이브러리 불러오기

In [1]:
import numpy as np
import pandas as pd

## 입력 데이터

In [2]:
import cv2
img = cv2.imread('./images/digits.png', cv2.IMREAD_GRAYSCALE)
print(img.shape)

(1000, 2000)


In [3]:
cells = [np.hsplit(row,100) for row in np.vsplit(img,50)]
x = np.array(cells) / 255.
X = x[:,:].reshape(-1,400).astype(np.float32)
y = np.repeat(np.arange(10),500)

In [4]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3,
                                                    random_state=1)

## 레이어 정의

In [5]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(512,256,128), verbose=True)

In [6]:
model.fit(train_X, train_y)

Iteration 1, loss = 1.27650205
Iteration 2, loss = 0.40096806
Iteration 3, loss = 0.26288954
Iteration 4, loss = 0.18236015
Iteration 5, loss = 0.15437636
Iteration 6, loss = 0.12544782
Iteration 7, loss = 0.09304241
Iteration 8, loss = 0.06756976
Iteration 9, loss = 0.05249714
Iteration 10, loss = 0.04338246
Iteration 11, loss = 0.03620788
Iteration 12, loss = 0.02651813
Iteration 13, loss = 0.02263116
Iteration 14, loss = 0.01635824
Iteration 15, loss = 0.01321957
Iteration 16, loss = 0.01043815
Iteration 17, loss = 0.00796769
Iteration 18, loss = 0.00602642
Iteration 19, loss = 0.00469493
Iteration 20, loss = 0.00417415
Iteration 21, loss = 0.00353307
Iteration 22, loss = 0.00310852
Iteration 23, loss = 0.00273591
Iteration 24, loss = 0.00241279
Iteration 25, loss = 0.00220898
Iteration 26, loss = 0.00203045
Iteration 27, loss = 0.00183984
Iteration 28, loss = 0.00171544
Iteration 29, loss = 0.00158643
Iteration 30, loss = 0.00148481
Iteration 31, loss = 0.00137980
Iteration 32, los

MLPClassifier(hidden_layer_sizes=(512, 256, 128), verbose=True)

In [7]:
import pickle
with open("number.model", "wb") as f:
    pickle.dump(model, f)

In [8]:
img4 = cv2.imread('./images/four.png', 0)
cv2.imshow("4", img4)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
number = img4.reshape(-1, 400)

import pickle
with open("number.model", "rb") as f:
    model = pickle.load(f)

print(model.predict_proba(number))
print(model.predict(number))

[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
[4]


In [10]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)
if cap.isOpened():
    while True:
        ret, img = cap.read()
        if not ret: break 

        g_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, bin_img = cv2.threshold(g_img, 110, 255, cv2.THRESH_BINARY_INV)
        contours, hierarchy = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, 
                                               cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours: 
            (x,y), radius = cv2.minEnclosingCircle(contour)
            if radius < 5: continue # 5보다 작은 객체 Skip 
            xs, xe = int(x-radius), int(x+radius)
            ys, ye = int(y-radius), int(y+radius)
            cv2.rectangle(bin_img, (xs,ys), (xe,ye), (200,0,0), 1)
            roi = bin_img[ys:ye, xs:xe]
            if roi.shape[0]*roi.shape[1] == 0: continue 
            roi = cv2.resize(roi, dsize=(16, 16))
            A = np.zeros((20,20))
            A[2:-2,2:-2] = roi[:,:]
            A = A.reshape(-1,400)
            num = model.predict(A)
            cv2.putText(bin_img, str(num), (xs, ys), cv2.FONT_HERSHEY_PLAIN, 2, (200,0,0))
        cv2.imshow("Image", bin_img)
        if cv2.waitKey(1)&0xFF == 27: break # ESC            
else :
    print("Camera not opened")
    
cap.release()
cv2.destroyAllWindows()

In [11]:
# RandomForestClassifier를 이용해서 모델을 만들 수 있습니다.
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(train_X, train_y)

RandomForestClassifier()

In [12]:
model.score(test_X, test_y)

0.938